In [21]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
crawler = NewsCrawler()

# crawler = NewsCrawler(
#     'americanconservative', 
#     'thehill',
#     'thenextweb',
#     'telegraph',
#     'indiatimes',
#     'theverge',
#     'usatoday',
#     'chinadaily',
#     'scmp',
#     'japan-news',
#     'japantoday',
#     'chosun',
#     'joongang',
#     'arirang',
#     'ccn',
#     'cointelegraph',
#     'cryptocoin',
#     'cryptonews')

# 1. URL을 모은다

In [23]:
# nextbigfuture, nbcnews는 필요없을지도...

In [24]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (239.42 seconds): Collecting URLs... rt                  

n_total     26002
n_unique    25933
dtype: int64

,pubs,actual_pub
https://abcnews.go.com/Politics/top-diplomat-ukraine-crazy-withhold-security-sasistance-political/story?id=66039011,"abcnews, americanconservative",abcnews
https://axios.com/trump-blamed-rick-perry-call-ukraine-zelensky-8178447a-0374-4ac6-b321-a9454b0565d4.html,"dailybeast, axios",axios
https://businessinsider.co.za/chicken-lickens-secret-menu-comes-in-a-black-box-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/india-pakistan-nuclear-war-death-famine-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/le-creuset-star-wars-collection-2019-10-2,"businessinsider, news24",businessinsider
https://cnn.com/style/article/banksy-devolved-parliament-auction-final-sale-price/index.html,"cnn, axios",cnn
https://hotair.com/archives/jazz-shaw/2019/10/05/universal-basic-income-test-work,"townhall, hotair",hotair
https://msnbc.com/ali-velshi/watch/house-democrats-seek-ukraine-documents-from-pence-70641733767,"msnbc, nbcnews",msnbc
https://msnbc.com/am-joy/watch/house-democrats-seek-ukraine-documents-from-pence-70703685916,"msnbc, nbcnews",msnbc
https://msnbc.com/the-beat-with-ari/watch/swizz-beatz-on-working-with-beyonce-jay-z-drake-and-upending-the-art-world-70266437589,"msnbc, nbcnews",msnbc


huffpost                  70
cnn                      830
investing.com            918
politico                 240
time                      20
cnbc                     166
fox                      322
bbc                      285
businessinsider          560
morningstar               95
wsj                      102
nytimes                  101
guardian                 151
reuters                 1169
washingtontimes          459
washingtonpost           129
cbs                      405
marketwatch              132
atlantic                 112
vice                      17
npr                      862
newrepublic               21
yahoo                    396
independent              725
heritage                 288
zdnet                    384
townhall                 604
abcnews                   98
hotair                    85
cbc                      301
                        ... 
metro                    245
msnbc                    120
nationalreview           719
news24        

# 2. 모은 URL중 이미 받은 것은 버린다

In [25]:
select_results = crawler.select(); display(*select_results)

100.00% (33.79 seconds): Selecting URLs... cryptonews          

n_total     2653
n_unique    2653
dtype: int64

,pubs,actual_pub


huffpost                 12
cnn                      48
investing.com            68
politico                  7
time                      2
cnbc                     20
fox                      69
bbc                      26
businessinsider         118
morningstar               2
wsj                      16
nytimes                  18
guardian                 54
reuters                 209
washingtontimes          53
washingtonpost           46
cbs                      24
marketwatch               7
atlantic                  1
vice                      0
npr                      11
newrepublic               0
yahoo                   142
independent              53
heritage                  0
zdnet                    17
townhall                  8
abcnews                  17
hotair                   35
cbc                      36
                       ... 
metro                    94
msnbc                    15
nationalreview            3
news24                   79
techcrunch          

In [122]:
crawler.urls_selected['cryptonews']

{'https://ar.cryptonews.com/guides/can-i-create-my-own-cryptocurrency.htm',
 'https://ar.cryptonews.com/guides/how-is-the-price-of-bitcoins-set.htm',
 'https://ar.cryptonews.com/guides/how-to-choose-a-bitcoin-wallet.htm',
 'https://ar.cryptonews.com/guides/is-it-safe-to-use-bitcoin.htm',
 'https://ar.cryptonews.com/guides/what-can-i-buy-with-bitcoin.htm',
 'https://ar.cryptonews.com/news/altcoin-news',
 'https://ar.cryptonews.com/news/bitcoin-news',
 'https://ar.cryptonews.com/news/bitcoin-price-altcoins-drop-below-key-supports-4084.htm',
 'https://ar.cryptonews.com/news/bitcoin-price-altcoins-rebound-remains-attractive-to-sellers-4147.htm',
 'https://ar.cryptonews.com/news/bitcoin-price-and-altcoins-struggle-to-clear-key-resistances-4141.htm',
 'https://ar.cryptonews.com/news/bitcoin-price-fluctuates-around-usd-10k-altcoins-decline-4075.htm',
 'https://ar.cryptonews.com/news/bitcoin-price-struggles-to-recover-altcoins-turned-sell-on-r-4101.htm',
 'https://ar.cryptonews.com/news/bitcoi

# 3. 해당 URL의 뉴스를 다운로드한다

In [26]:
download_results = crawler.download(); download_results

24.50% (465.48 seconds): Downloading... guardian            

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100.00% (2366.81 seconds): Downloading... dailymail           

,downloaded,trashed
abcnews,17,0
afr,0,0
aljazeera,12,77
americanconservative,0,0
arirang,0,0
arstechnica,5,0
arynews,17,24
atlantic,1,0
axios,73,19
bbc,19,7


In [20]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['japan-news']['trashed']]

['d80494a4761ead4276e510dc4f64762e319902e0',
 'f25b1d06e58c05786ad57fe9bea9817859de3861',
 '02d668060004e25c2299c9a23445e38a9c6ec771',
 '2d3e330325b35d2c622806feac055478a4a61559',
 '2e87c621d1ffb7f28485496eb82814891ff5dd1b',
 '40cf0a5a2b87e040cd211da1855cb615702cb0ca',
 'feefa98b143ce76ef3cadf0a2764ca634f773fee',
 '0a900188f145b48aad8c70bde741aea563dba542',
 'ef7e33fa30c8be4b6d90681b383234ee4f73e51a',
 '64cc9a5e5a51520306154fda70d482b57a7420b0',
 '12e43a97340dc5a9f9270b38a9705e461456041b',
 'd71b4a23216b50deb276d3ccd8c1905939f5a9c2',
 '9a5e5f813637acd5359da986e562fa9ad9858f4b',
 '44c36bff1fe4d64fb468131f1221959fa30718d6',
 'dbf61f3dd462f9c650d0b2f42ca69717701fc7d1',
 '385dbc7ad54410ac2078055044e34c6e30cfde4b',
 'a31fe291585e194540805022e45df9ebdbabc9fc',
 '66afae226b2dc5d329fc731f7931f9f6d6f37e23']